# MATH50003 Numerical Analysis (2023–2024) Computer-based Exam

Instructions for uploading and downloading:

1. Rename the file to include your CID.
2. You have 15 mins to download the exam beginning at 9:00 on 15 March. **DO NOT** write or type anything during this time.
2. You have 1 hour to complete the exam beginning at 9:15 on 15 March. You **MUST STOP** typing at 10:15.
3. Deadline is 10:30 on 15 March to upload the completed Jupyter notebook (`.ipynb`) to Blackboard.
5. Once uploaded, re-download the file before the final submission time to confirm it is correct.
You are allowed to upload additional submissions but only the last valid upload before 10:30 will be used.
6. If uploading via Blackboard fails or please contact an invigilator.

Instructions for the exam:

1. For each problem, replace the `# TODO` to complete the question.
The unit tests are provided to help you test your answers, but do not guarantee that the answer is correct.
2. Problems are marked A/B/C to indicate difficulty ("A" being most difficult).
3. All questions are worth 10 marks. Partial credit will be awarded for reasonable attempts or comments outlining a solution even if the tests
are not passed.
3. If you have technical queries please contact an invigilator.
4. You may use existing code from anywhere
but you are **REQUIRED** to cite the source if it is not part of the module material,
by including a weblink in a comment.
5. You **MUST NOT** ask for help online or
communicate with others within or outside the module.
Failure to follow these rules will be considered misconduct.
6. **NO USAGE of AI tools** such as ChatGPT or GitHub Co-Pilot.
7. **TURN OFF notifications** on your machine. In particular, close background processes for any chat applications (iMessage, WhatsApp, WeChat).

You should use the following packages:

In [1]:
using LinearAlgebra, SetRounding, Test

**WARNING** It may be necessary to restart the kernel if issues arise. Remember to reload the packages
when you do so.

-----

**Problem 1 (C)** Implement the mid-point rule for approximating an integral:
$$
\int_0^1 f(x) {\rm d}x ≈ {1 \over n} \sum_{k=1}^n f(x_{k-1/2})
$$
where $x_k := k/n$. By computing with various values of $n$, conjecture what the convergence rate is as $n → ∞$,
that is, for what $λ$ is the error $O(n^{-λ})$? You may assume all numbers are `Float64`.

In [2]:
function midpointrule(f, n)
    # TODO: implement the mid-point rule
    # SOLUTION
    ret = 0.0 # 1 mark for setup
    # 2 marks for a similar loop
    for k = 1:n
        ret += f((k-1/2)/n) ## 4 marks for evaluating f at right points
    end
    ret/n
    # END
end

@test midpointrule(exp, 5000) ≈ (exp(1)-1)

# TODO: Replace 0 below to set λ to what you conjecture that the convergence rate O(n^(-λ)) is.
# Include any experiments/comments to justify your conjecture.
λ = 0
# SOLUTION
# 2 marks for justifying experiment (even if conclusion is wrong)
(midpointrule(exp, 100) - (exp(1)-1))/(midpointrule(exp, 200) - (exp(1)-1)) # ≈ 4
# This means second order since (2n)^2 ≈ 4n^2.
λ = 4 ## 1 mark for correct answer
# END

4

**Problem 2 (B)** Derive a third-order divided difference formula by combining a first-order and second-order divided difference, that
evaluates at the points $f(x+2h), f(x+h), f(x),$ and $f(x-h)$.
Implement this formula in the following function:

In [3]:
function thirdorderdivideddifference(f, x, h)
    # TODO: approximate f'''(x) by deriving a third-order divided difference formula
    # SOLUTION
    # 3 marks for similar expression
    # We have f''(x) ≈ (f(x+h) - 2f(x) + f(x-h))/h^2 hence
    # f'''(x) ≈ (f'(x+h) - 2f'(x) + f'(x-h))/h^2
    #         ≈ (f(x+2h) - 3f(x+h) + 3f(x) - f(x-h))/h^3
    (f(x+2h) - 3f(x+h) + 3f(x) - f(x-h))/h^3
    # END
end

# We test with BigFloat as the round-off errors in third-order divided differences are too large to get accurate digits:
@test thirdorderdivideddifference(x -> cos(x^2), big(1.0), big(0.0000001)) ≈ -12cos(1) + 8sin(1) atol=1E-5

Test Passed

**Problem 3 (C)** Add support for `tan` and `acos` (which implements $\arccos(x)$)  with duals. You may use the provided `cos` and `sin`
implementations. Hint: recall that $\arccos'(x) = -1/\sqrt{1-x^2}$.

In [4]:
struct Dual
    a
    b
end

import Base: tan, acos, cos, sin, /


cos(x::Dual) = Dual(cos(x.a), -sin(x.a) * x.b)
sin(x::Dual) = Dual(sin(x.a), cos(x.a) * x.b)
function /(x::Dual, y::Dual)
    if iszero(y.a)
        error("Division for dual numbers is ill-defined when denonimator real part is zero.")
    end
    return Dual(x.a / y.a, (y.a * x.b - x.a * y.b) / y.a^2)
end

function tan(x::Dual)
    # TODO: implement tan for dual numbers
    # SOLUTION
    # 5 marks. Using explicit formula for tan'(θ) == sec(θ)^2 also allowed.
    sin(x)/cos(x)
    # END
end

function acos(x::Dual)
    # TODO: implement acos, i.e. arccos, for dual numbers
    # SOLUTION
    # 5 marks
    a,b = x.a, x.b
    Dual(acos(a), -b/sqrt(1-a^2))
    # END
end

@test (tan(acos(Dual(0.1,1)))/cos(Dual(0.1,1))).a ≈ tan(acos(0.1))/cos(0.1)
@test (tan(acos(Dual(0.1,1)))/cos(Dual(0.1,1))).b ≈ -((sec(0.1)*(1 + 0.1*(0.1^2-1)*tan(0.1)))/(0.1^2*sqrt(1 - 0.1^2)))

Test Passed

**Problem 4 (C)** A `Float16` represents a number of the form $2^{q-15} (1.b_1…b_{10})_2$.
 Create a function that returns the exponent $q$ of a `Float16` as an `UInt8`. Do not use the
built-in `exponent` function. Hint: combine `bitstring` and `parse`.

In [5]:
function floatexponent(x::Float16)
    # TODO: return the exponent of x
    # SOLUTION
    # 2 marks for using `bitstring` and 2 marks for using `parse`
    # Other solutions are permitted if correct.
    parse(UInt8, bitstring(x)[2:6]; base=2)
    # END
end


@test floatexponent(Float16(0.0123)) isa UInt8
@test floatexponent(Float16(0.0123)) == 8

Test Passed

**Problem 5.1 (C)** Complete the implementation of a function `cos_t(x,n)` computing the
first `2n` terms of the Taylor series:
$$
\cos\ x ≈ ∑_{k=0}^n {(-1)^k x^{2k} \over (2k)!}
$$

In [6]:
function cos_t(x, n)
    ret = x^2
    s = x^2
    # TODO: Compute the first 2n terms of the Taylor series of cos,
    # without using the factorial function.
    # SOLUTION
    # 3 marks for looping 1:n
    for k = 1:n
        s = -s/((2k-1)*(2k)) * x^2 ## 2 marks for updating s and 1 mark for x^2
        ret = ret + s
    end
    ret
    # END
end

@test cos_t(1.0, 10) ≈ cos(1)

Test Passed

**Problem 5.2 (B)**  Complete the implementation of a function `cos_bound(x,n)` that
includes an error bound on the computation. You may assume $0 ≤ x ≤ 1$.

In [7]:
struct Interval # represents the set [a,b]
    a # left endpoint
    b # right endpoint
end

Interval(x) = Interval(x,x) # Support Interval(1) to represent [1,1]

import Base: *, +, -, ^, /, one, in

in(x, X::Interval) = X.a ≤ x ≤ X.b
one(X::Interval) = Interval(one(X.a), one(X.b))
function +(X::Interval, Y::Interval)
    a,b,c,d = promote(X.a, X.b, Y.a, Y.b) # make sure all are the same type
    T = typeof(a)
    α = setrounding(T, RoundDown) do
        a + c
    end
    β = setrounding(T, RoundUp) do
        b + d
    end
    Interval(α, β)
end
function -(X::Interval)
    a,b = promote(X.a, X.b)
    Interval(-b, -a)
end
function /(X::Interval, n::Int)
    a,b = promote(X.a, X.b)
    T = eltype(a)
    if n == 0
        error("Dividing by zero not support")
    end
    α = setrounding(T, RoundDown) do
        n > 0 ? a / n : b / n
    end
    β = setrounding(T, RoundUp) do
        n > 0 ? b / n : a / n
    end
    Interval(α, β)
end
function *(X::Interval, Y::Interval)
    a,b,c,d = promote(X.a, X.b, Y.a, Y.b)
    T = typeof(a)
    if !(a ≤ b && c ≤ d)
        error("Empty intervals not supported.")
    end
    α = setrounding(T, RoundDown) do
        min(a*c,a*d,b*c,b*d)
    end
    β = setrounding(T, RoundUp) do
        max(a*c,a*d,b*c,b*d)
    end
    Interval(α, β)
end
function ^(X::Interval, k::Int)
    if k ≤ 0
        error("not supported")
    elseif k == 1
        X
    else
        X * X^(k-1)
    end
end


function cos_bound(X::Interval, n)
    a,b = promote(X.a, X.b)
    T = typeof(a)
    if !(0 ≤ a ≤ b ≤ 1)
        error("Interval must be a subset of [0, 1]")
    end
    # TODO: complete the implementation of cosine applied to an interval,
    # including the error in truncating the Taylor series.
    # SOLUTION
    # 2 mark for calling cos_t
    ret = cos_t(X, n)
    # 2 mark for constructing factorial, 2 marks for getting rounding correct
    f = T(factorial(big(2n+2)),RoundDown)

    err = setrounding(T, RoundUp) do
        T(1) / f # 3 mark6
    end
    ret + Interval(-err,err) # 3 marks
    # END
end

@test cos(big(1)) in cos_bound(Interval(1.0), 10)
@test cos_bound(Interval(1.0), 10).b - cos_bound(Interval(1.0), 10).a ≤ 1E-14

Test Passed

**Problem 6 (B)** Complete the implementation of `LowerTridiagonal` which represents an $n × n$ banded matrix with
bandwidths $(l,u) = (2,0)$ by storing only its diagonal, sub-diagonal, and second-sub-diagonal as vectors.
Overload `setindex!`, as well as `\` to solve a lower tridiagonal system in $O(n)$ operations. For simplicity you may assume all numbers are `Float64`.

In [8]:
import Base: getindex, setindex!, size, \

struct LowerTridiagonal <: AbstractMatrix{Float64}
    d::Vector{Float64}   # diagonal entries of length n
    dl::Vector{Float64}  # sub-diagonal entries of length n-1
    dl2::Vector{Float64} # second-sub-diagonal entries of length n-2
end

size(L::LowerTridiagonal) = (length(L.d),length(L.d))

# implements L[k,j]
function getindex(L::LowerTridiagonal, k::Int, j::Int)
    d, dl, dl2 = L.d, L.dl, L.dl2
    if k == j
        d[k]
    elseif k == j+1
        dl[j]
    elseif k == j+2
        dl2[j]
    else
        0.0
    end
end

# implements L[k,j] = v
function setindex!(L::LowerTridiagonal, v, k::Int, j::Int)
    d, dl, dl2 = L.d, L.dl, L.dl2
    if j > k || j < k-2
        error("Cannot modify off-band")
    end
    # TODO: set L[k,j] to be equal to v
    # SOLUTION
    # 3 marks, give 1 mark if different cases considered
    if k == j
        d[k] = v
    elseif k == j+1
        dl[j] = v
    elseif k == j+2
        dl2[j] = v
    end
    # END
end

function \(L::LowerTridiagonal, b::AbstractVector)
    m,n = size(L) # we know m == n
    if n ≠ length(b)
        error("Dimensions do not match")
    end
    if n < 1
        error("Empty vectors not supported")
    end
    x = zeros(n) # Vector to be populated
    # TODO: Populate the vector x so that L*x ≈ b, using O(n) operations.
    # SOLUTION
    # 7 marks, only give 4 marks if not O(n)
    x[1] = b[1]/L[1,1]
    # only take off 1 mark for forgetting special case
    if n ≥ 2
        x[2] = (b[2]-L[2,1]*x[1])/L[2,2]
    end
    for k = 3:n
        x[k] = (b[k] - L[k,k-1]*x[k-1] - L[k,k-2]*x[k-2])/L[k,k]
    end
    x
    # END
end

n = 10
d, dl, dl2 = randn(n), randn(n-1), randn(n-2)
L = LowerTridiagonal(d, dl, dl2)
@test L == diagm(0 => d, -1 => dl, -2 => dl2)

L[3,2] = 4
@test L[3,2] == 4

b = randn(n)
@test L\b ≈ Matrix(L)\b

Test Passed

**Problem 7 (A)**  Consider the second-order initial value problem
$$
u(0) = 1, u'(0) = 0, u''(x) = x u(x)
$$
for $x ∈ [0,1]$. Recast this problem as a linear system involving a `LowerTridiagonal` matrix (as defined in Problem 6)
using a finite-difference approximation on the evenly grid $x_0,x_1,…,x_n$
where $x_k = k/n$ using the right-sided divided difference to approximate
$$
u'(0) ≈ {u(h) - u(0) \over h}
$$
and the second-order divided difference approximation
$$
u''(x) ≈ {u(x+h) - 2u(x) + u(x-h) \over h^2}
$$
imposed at the grid points $x_1,…,x_{n-1}$.

In [9]:
function secondorderairysystem(n)
    # TODO: return a LowerTridiagonal matrix representing the differential operator u'' - x*u,
    # with the first two rows representing the initial conditions u(0) and u'(0)
    # SOLUTION
    x = range(0,1; length=n+1) # 2 marks for setting up x
    h = step(x) # 1 mark for computing h (h = 1/n also allowed)
    # 5 marks for correct answer. Give 2 marks if similar but incorrect
    LowerTridiagonal([1; 1/h; fill(1/h^2, n-1)],
                     [-1/h; -x[2:end-1] .- 2/h^2],
                     fill(1/h^2, n-1))
    # END
end

function secondorderairyrhs(n)
    # TODO: return the right-hand side corresponding to the above system.
    # SOLUTION
    # 2 marks
    [1; zeros(n)]
    # END
end

n = 1000
@test secondorderairysystem(n) isa LowerTridiagonal
u = secondorderairysystem(n) \ secondorderairyrhs(n)
@test u[end] ≈ 1.1722999700579313 atol=1E-5

Test Passed

**Problem 8.1 (B)** Implement the following function that given a real vector $𝐱$ will return a 2 × 2 rotation matrix such that
$$
Q 𝐱 = \| 𝐱 \| 𝐞_2.
$$
You may assume all numbers are `Float64`.

In [10]:
function rotatedown(x)
    if length(x) ≠ 2
        error("Length must be 2")
    end
    # TODO: Create a 2 × 2 rotation matrix Q such that Q*x = [0,norm(x)].
    # SOLUTION
    # Note another solution would compute the angle. Give 4 marks
    # for reasonable attempts.
    a,b = x
    [b -a; a b]/sqrt(a^2 + b^2)
    # END
end

x = randn(2)
Q = rotatedown(x)
@test Q'Q ≈ I
@test det(Q) ≈ 1
@test Q*x ≈ [0,norm(x)]

Test Passed

**Problem 8.2 (A)** Implement a QL factorisation for a tridiagonal matrix, returning an orthogonal matrix `Q`
and `L`  represented as a `LowerTridiagonal` matrix, as defined in Problem 6. You may assume all numbers are `Float64`. Hint: unlike the `bandedqr` implemented in the labs, `Q` may be stored as
as a dense matrix. Because of this your solution will likely take $O(n^2)$ operations.

In [11]:
function bandedql(A::Tridiagonal)
    n = size(A, 1)
    Q = Matrix(1.0I, n, n) # create an n × n identity matrix
    L = LowerTridiagonal(zeros(n), zeros(n - 1), zeros(n - 2))

    # TODO: Populate Q and L so that Q*L ≈ A in at most O(n^2) operations.

    # SOLUTION
    # 1 mark for initial step
    L[n, n-1:n] = A[n, n-1:n]
    # 2 marks for recognising the columns need to go backwards.
    for j = n:-1:2
        # 2 marks for recognising that rotatedown can be used.
        x_1 = [A[j-1, j], L[j, j]]
        Q_1 = rotatedown(x_1)
        L[j,j] = (Q_1 * x_1)[2]

        x_2 = [A[j-1, j-1], L[j, j-1]]
        L[j-1:j,j-1] = Q_1*x_2

        # 1 mark for remembering special case
        if j > 2
            x_3 = [A[j-1, j-2], L[j, j-2]]
            L[j-1:j,j-2] = Q_1*x_3
        end

        # 2 marks for updating Q
        Q[:,j-1:j] = Q[:,j-1:j]*Q_1'
    end
    # END
    Q, L
end

n = 100
A = Tridiagonal(randn(n-1), randn(n), randn(n-1))
Q,L = bandedql(A)

@test Q'*Q ≈ I
@test L isa LowerTridiagonal
@test Q*L ≈ A

Test Passed

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*